In [1]:
# This is broken, unclear how to fix

#!/usr/bin/env python3
import os, json, time, requests
from datetime import date, datetime, timedelta

# --------------------------- original variables ----------------------------
output_file = "thunderhead-raw-data/daily-unique-users.json"
start_date  = "2025-01-01"
end_date    = date.today().isoformat()

gql_url  = "https://hyperliquid.thunderhead.xyz/graphql"
# use the FastAPI‐served endpoint on the same host as GraphQL:
rest_url = "https://hyperliquid.thunderhead.xyz/hyperliquid/daily_unique_users"

query = """
query DailyUsers($from: ISODate!, $to: ISODate!) {
  dailyUniqueUsers(startDate: $from, endDate: $to) {
    time
    daily_unique_users
  }
}
"""
# ---------------------------------------------------------------------------

headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/122.0.0.0 Safari/537.36"),
    "Content-Type": "application/json"
}

results = []

def month_end(d: date) -> date:
    nxt = d.replace(day=28) + timedelta(days=4)
    return nxt - timedelta(days=nxt.day)

def fetch_from_graphql(start_iso: str, end_iso: str) -> list[dict]:
    """Try GraphQL (with potential 522 retries)."""
    payload = {"query": query, "variables": {"from": start_iso, "to": end_iso}}
    r = requests.post(gql_url, headers=headers, json=payload, timeout=30)
    if r.status_code >= 500:
        # Cloudflare 522, etc.
        raise requests.HTTPError(f"{r.status_code} upstream")  # 
    r.raise_for_status()
    data = r.json()
    if "errors" in data:
        raise ValueError(data["errors"])
    return data["data"]["dailyUniqueUsers"]

def fetch_from_rest(start_iso: str, end_iso: str) -> list[dict]:
    """Fallback via FastAPI endpoint (returns {'chart_data':[...]})."""
    r = requests.get(
        rest_url,
        params={"start_date": start_iso, "end_date": end_iso},
        headers=headers,
        timeout=30
    )
    r.raise_for_status()
    return r.json()["chart_data"]                       # :contentReference[oaicite:0]{index=0}

cursor = datetime.fromisoformat(start_date).date()
final  = datetime.fromisoformat(end_date).date()

while cursor <= final:
    seg_end = min(month_end(cursor), final)
    start_iso, end_iso = cursor.isoformat(), seg_end.isoformat()

    delay, tries, done = 1, 0, False
    while not done and tries < 6:
        try:
            chunk = fetch_from_graphql(start_iso, end_iso)
            results.extend(chunk)
            done = True
        except (requests.RequestException, ValueError) as err:
            tries += 1
            if tries >= 6:
                print(f"[warn] GraphQL down for {start_iso}→{end_iso}: {err}. Falling back to REST.")
                try:
                    results.extend(fetch_from_rest(start_iso, end_iso))
                except requests.HTTPError as e2:
                    print(f"[warn] REST also failed for {start_iso}→{end_iso}: {e2}. Skipping this segment.")
                done = True
            else:
                time.sleep(delay)
                delay *= 2

    cursor = seg_end + timedelta(days=1)

os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ Saved {len(results):,} rows → {output_file}")


[warn] GraphQL down for 2025-01-01→2025-01-31: 522 upstream. Falling back to REST.
[warn] REST also failed for 2025-01-01→2025-01-31: 522 Server Error: <none> for url: https://hyperliquid.thunderhead.xyz/hyperliquid/daily_unique_users?start_date=2025-01-01&end_date=2025-01-31. Skipping this segment.
[warn] GraphQL down for 2025-02-01→2025-02-28: 522 upstream. Falling back to REST.
[warn] REST also failed for 2025-02-01→2025-02-28: 522 Server Error: <none> for url: https://hyperliquid.thunderhead.xyz/hyperliquid/daily_unique_users?start_date=2025-02-01&end_date=2025-02-28. Skipping this segment.
[warn] GraphQL down for 2025-03-01→2025-03-31: 522 upstream. Falling back to REST.
[warn] REST also failed for 2025-03-01→2025-03-31: 522 Server Error: <none> for url: https://hyperliquid.thunderhead.xyz/hyperliquid/daily_unique_users?start_date=2025-03-01&end_date=2025-03-31. Skipping this segment.
[warn] GraphQL down for 2025-04-01→2025-04-19: 522 upstream. Falling back to REST.
[warn] REST al